In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
%config Completer.use_jedi = False
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/zomato-bangalore-dataset/zomato.csv")
df.head()

In [ ]:
df.shape

# Problem Statemet 1
## Getting all NaN Features

In [ ]:
df.isnull().sum()

In [ ]:
#iterate through columns
naFeature = [feature for feature in df.columns if df[feature].isnull().sum() > 1]
naFeature

```withoutt list comprehension```
naCol = []

for feature in df.columns:

    if df[feature].isnull().sum()>1:
    
        naCol.append(feature)
        
        
naCol


In [ ]:
#print the missing percentage
# df['rate'].isnull().sum()/len(df) * 100
for feature in naFeature:
    print('{} has {} % missing value. '.format(feature,np.round(df[feature].isnull().sum()/len(df) * 100 ,2)))

In [ ]:
df.info()

## Data CLeaning

In [ ]:
df['approx_cost(for two people)'].unique()

In [ ]:
#change the data type of 'approx_cost'
#1. change into str to replace the ',' values
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype('str').apply(lambda x:x.replace(',',''))

In [ ]:
# #without lambda
# def removeComma(x):
#     return x.replace(',','')

# df['approx_cost(for two people)'].astype(str).apply(removeComma)

In [ ]:
df['approx_cost(for two people)'].unique()

In [ ]:
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(float)
df['approx_cost(for two people)'].dtype    #data is change into float

In [ ]:
#let's change the dtype of rate
df['rate'].dtype

In [ ]:
df['rate'].unique()

So we have to remove the '/' to convert it into flot or int

In [ ]:
df['rate'].isnull().sum()

call the split and split it on the basisi of '/' and return the 0th index

In [ ]:
df['rate'][0].split('/')[0]

In [ ]:
def split(x):
    return x.split('/')[0]

In [ ]:
df['rate'] = df['rate'].astype(str).apply(split)

In [ ]:
df['rate'].unique()

we have to replace the special character '-' and 'NEW' with 0

In [ ]:
df['rate'].replace('-', 0, inplace=True)
df['rate'].replace('NEW', 0, inplace=True)

In [ ]:
df['rate'] = df['rate'].astype(float)
df['rate'].dtype

In [ ]:
df.info()


# Problem Statement
## How many types of Restaurant do we have?

In [ ]:
#plot the top 20 entries
plt.figure(figsize=(20,12))
df['rest_type'].value_counts().nlargest(20).plot.bar(color='green')

In [ ]:
# make this in summarazie way
def summ_mark(x):
    '''if the x belongs to QUick Bites and Casual Dining'''
    
    if x in ('Quick Bites', 'Casual Dining'):
        return 'Quick Bites + Casual Dining'
    else:
        return 'others'

In [ ]:
#apply this function in rest type
df['top_types']=df['rest_type'].apply(summ_mark)

In [ ]:
df.head(5)

In [ ]:
#we use plotly
import plotly.express as px

In [ ]:
values = df['top_types'].value_counts().values

In [ ]:
labels = df['top_types'].value_counts().index

In [ ]:
fig=px.pie(df, names=labels, values=values, title='Restaurant Type')
fig.show()

# Problem Statement
## Create a new DF for Votes, Cost and Rating of Each Restaurant

**Group your data on the basis of name of restaurant**

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
restaurant = df.groupby('name').agg({'votes':'sum',
                        'url':'count', 
                        'approx_cost(for two people)': 'mean',
                        'rate': 'mean'
                       }).reset_index()
restaurant

In [ ]:
restaurant.columns = ['name', 'totalVotes', 'totalUrls', 'avgApproxCost', 'avgRating']
restaurant.head()

In [ ]:
# caclulate total votes per total urls

restaurant['votesPerUrl'] = restaurant['totalVotes']/restaurant['totalUrls']
restaurant.head()

In [ ]:
# Creating Data Frame to sort the restaurant based on unitites
popularRest = restaurant.sort_values(by='totalUrls', ascending=False)
popularRest

In [ ]:
popularRest.shape

Now i can showcase important informations
1. Average Votes received by restaurant
2. Top 5 most Voted Restaurant
3. Top 5 less Voted Restaruant


In [ ]:
popularRest.sort_values(by='totalVotes', ascending=False).query('totalVotes > 0')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20,25))
ax1.text(0.50, 0.30, int(popularRest['totalVotes'].mean()), fontsize=45, ha = 'center')
ax1.text(0.50, 0.12, 'is the average of votes', fontsize=22, ha='center')
ax1.text(0.50, 0.05, 'recieved by restaurants', fontsize=22, ha='center')
ax1.axis('off')
sns.barplot(x='totalVotes', y='name', data = popularRest.sort_values(by='totalVotes', ascending=False).query('totalVotes > 0').head(5), ax = ax2)
ax2.set_title('Top 5 most Voted Restaurant')

sns.barplot(x='totalVotes', y='name', data = popularRest.sort_values(by='totalVotes', ascending=False).query('totalVotes > 0').tail(5), ax = ax3)
ax3.set_title('Top 5 least Voted Restaurant')


# Problem Statemnts
1. ANalyise most expenisve and most chepest restaurants in bangalore

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20,25))
ax1.text(0.50, 0.30, int(popularRest['avgApproxCost'].mean()), fontsize=45, ha = 'center')
ax1.text(0.50, 0.12, 'is the mean approx cost', fontsize=22, ha='center')
ax1.text(0.50, 0.05, 'in Bangalore City', fontsize=22, ha='center')
ax1.axis('off')
sns.barplot(x='avgApproxCost', y='name', data = popularRest.sort_values(by='avgApproxCost', ascending=False).query('avgApproxCost > 0').head(5), ax = ax2)
ax2.set_title('Top 5 most Expensive Restaurant')

sns.barplot(x='avgApproxCost', y='name', data = popularRest.sort_values(by='avgApproxCost', ascending=False).query('avgApproxCost > 0').tail(5), ax = ax3)
ax3.set_title('Top 5 Chepest Restaurant')

## 2. How many table offer booking table Service?

In [ ]:
x = df['book_table'].value_counts()
labels = ['not book', 'book']

In [ ]:
import plotly.graph_objs as pgo
from plotly.offline import iplot

In [ ]:
trace = pgo.Pie(labels=labels, values= x, hoverinfo='label+percent', textinfo='percent', title='Table Booking')
iplot([trace])

## 3. Check which restaurants has online order service?

In [ ]:
x = df['online_order'].value_counts()
labels = ['Online Accepted', 'Online Not Accepted']

In [ ]:
onlineOrder = pgo.Pie(labels=labels, values= x, hoverinfo='label+percent', textinfo='percent', title="Accpetance of Online Order")
iplot([onlineOrder])

# Find Budget Friendly Restaurant!
ie Avg Cost for two people < 400

In [ ]:
filterQuery = (df['approx_cost(for two people)']<=400) & (df['location'] == 'BTM') & (df['rate']>=4) & (df['rest_type'] == 'Quick Bites')
budget = df[filterQuery]
budget.head()

In [ ]:
# we need only name of restaurant
budget['name'].unique()

> **Create the function for budget friendly restaurant**

In [ ]:
# so lets create a function to return budget retaurant
def budgetRestaurant(location, restaurantType):
    '''This funciton will help you to get the budget friendly Restaurant at your location in Bangalore
    Input: Location, Restaurant Type
    Return: List of budget friendly Restaurant'''
    
    filterQuery = (df['approx_cost(for two people)']<=400) & (df['location'] == location) & (df['rate']>=4) & (df['rest_type'] == restaurantType)
    budget= df[filterQuery]
    return budget['name'].unique

# Extract Latitutde and longitude of any place of restaurant

*The need of Lat and Long is to know the places where we get the budget Restaurant*

**For this we need *geopy* to plot the locations on the map**

In [ ]:
from geopy.geocoders import Nominatim

#Nomintim is the search engine for the location street data
#It will return the latitude and longitutde of the location

In [ ]:
geoLocator = Nominatim(user_agent='app')


In [ ]:
#lets get latitude and longitude of first row
data=df['name'][0]
data

here we are getting string formated values. 
- TO get lat and long we need to convert in some geographical format

In [ ]:
data = geoLocator.geocode(data)
print("{} is the Latitude and {} is the longitude.".format(data.latitude, data.longitude))

In [ ]:
locationsData = pd.DataFrame({'Name': df['location'].unique()})
locationsData.head()

In [ ]:
#we have to extract latitude and longitude of each and every location
lat = []
lon = []
for location in locationsData['Name']:
    location = geoLocator.geocode(location)
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)

In [ ]:
locationsData['latitude'] = lat
locationsData['longitude'] = lon

In [ ]:
locationsData.head()

# Perform Geolocation Analysis
## to know about where restaurants are situated

ie count of restaurant at a perticular location


In [ ]:
restaurantLoc = df['location'].value_counts().reset_index()
restaurantLoc

In [ ]:
#manipulate the column name
restaurantLoc.columns = ['Name', 'Count']
restaurantLoc

In [ ]:
#Merge the two df ie restaurantLoc and locationData
locationofRest= restaurantLoc.merge(locationsData, on='Name').dropna()
locationofRest

In [ ]:
# !pip install folium

import folium
from folium.plugins import HeatMap

In [ ]:
baseMap = folium.Map(location=[12.97, 77.59])    #lataitude and longitude of bangalore
baseMap

In [ ]:
HeatMap(data = locationofRest[['latitude','longitude', 'Count']]).add_to(baseMap)

In [ ]:
baseMap

# Task: Most Liked Dishes by Customers
### What Customers like what they dislike

For this we use **wordcloud** --> the bigger the world will be,  most of the customers will like it


In [ ]:
# !pip install wordcloud
from wordcloud import WordCloud, STOPWORDS

#perform on Quick Bites Resturant
data = df[df['rest_type'] == 'Quick Bites']
data.head()

#### Handle the missing values in dishliked column

In [ ]:
data['dish_liked'].isnull().sum()

In [ ]:
# we have to consider only the 'dish_liked' column 
dishes = ''   #empty string
for word in data['dish_liked'].dropna():     #dropna is to drop the null value
    words = word.split()   #it  return the list and store in words
    #iterate on the list
    for i in range(len(words)):
        #convert each and every words into lower case to make every letter looks same in the lower case (we can also choose uppercase)
        words[i] = words[i].lower()
    #convert list into string we have to perform join
    dishes = dishes + ' '.join(words) + ' '

In [ ]:
dishes[1:100]

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, width =1500, height = 1500).generate(dishes)

In [ ]:
plt.figure(figsize=(12,8))

plt.imshow(wordcloud)
plt.axis('off')

# Task: Reviews of Customers -> Sentiment Analysis

In [ ]:
data = df['reviews_list'][0]
data

In [ ]:
data = data.lower()
data

In [ ]:
import re
data2= re.sub('[^a-zA-Z]', ' ', data)
data2

In [ ]:
data3 = re.sub('rated', ' ', data2)
data3

In [ ]:
data4 = re.sub('x', ' ', data3)
data4

In [ ]:
#remove extra spaces
data5 = re.sub(' +', ' ', data4)
data5

In [ ]:
#now our data is ready for sentiment analysis
dataset = df[df['rest_type'] == 'Quick Bites']
dataset

In [ ]:
totalReview = ''
for review in dataset['reviews_list']:
    review = review.lower()   #lower reivew
    review  = re.sub('[^a-zA-Z]', ' ', review)
    review = re.sub('rated', ' ', review)
    review = re.sub('x', ' ', review)
    review = re.sub(' +', ' ', review)
    #concatenate
    totalReview = totalReview + str(review)

In [ ]:
wordcloudReview = WordCloud(stopwords=stopwords, width =1500, height = 1500).generate(totalReview)
plt.figure(figsize=(12,8))
plt.imshow(wordcloudReview)
plt.axis('off')

# Preparation of data for ML ALgo


In [ ]:
df.head()

In [ ]:
# split the dataframe in two categories --> 
#1. newRest --> rating ==0
#2. trainValRest --> rating > 0  [for trainig]

def assign(x):
    """Split the data in two categories"""
    if x>0:
        return 1
    else: 
        return 0

In [ ]:
df['rated'] = df['rate'].apply(assign)
df['rated'].head()

In [ ]:
df['rated'].unique()

In [ ]:
newRest = [df['rated'] == 0]
trainValRest = df.query("rated==1")

In [ ]:
trainValRest.head()

In [ ]:
trainValRest['rate'].unique() #restaurant with rating >0

In [ ]:
df.columns

**Now we have to create a feature for whether a restaurant is going to success or not!**

*Domain Expertise tells us which restaurant will be in which category*
1. rest < 3.75 --> bad(0)
2. if rest >3.75 --> good(1)

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
trainValRest['target'] = trainValRest['rate'].apply(lambda x: 1 if x >= 3.75 else 0)

In [ ]:
trainValRest.head()

In [ ]:
# check the label distribution of target!

x = trainValRest['target'].value_counts()
labels = x.index
print(x)
print("Indexes: ", labels)

In [ ]:
#plot this
plt.pie(x, labels = labels)

# Perform Feature Importance on data